In [1]:
# Read Data,Import Packages,Set Parameters & Wrangling

In [2]:
import pandas as pd
from bson import ObjectId
from pymongo import MongoClient
from datetime import datetime, timedelta, timezone
import numpy as np
from tifffile.tifffile import rational

In [3]:
# for Querying MongoDB
beginning_date = datetime(2024, 9, 21, 20, 30)
finishing_date = datetime.now()
module_name = "Onlineshopping"

In [12]:
df_orders = pd.read_parquet('df_orders.parquet')
df_min_order_date = pd.read_parquet('df_min_order_date.parquet')
df_dim_date = pd.read_parquet("dimdate5.parquet") # to attach shamsi date
rfm_labels = pd.read_excel("OnlineShoppingLabelsUpdate.xlsx", engine='openpyxl') # RFM Label for any modules

In [5]:
# Set Parameters Before Run --- old version
start_train_date_filter = pd.to_datetime('2024-09-22')
window_table_in_months = 3
# Define cluster categories
low_valued_clusters_labelFa = ['مشتریان جدید', 'مشتریان جدید در معرض خطر', 'مشتریان جدید و ارزشمند در آستانه ریزش', 'مشتریان کم ارزش']
high_valued_clusters_labelFa = ['مشتریان ارزشمند', 'مشتریان پتانسیل ارزشمند', 'مشتریان جدید با پتانسیل رشد', 'مشتریان ارزشمند یا پتانسیل ارزشمند و در معرض خطر', 'مشتریان جدید یا جدید ارزشمند و در معرض خطر', 'مشتریان ارزشمند در آستانه ریزش']

In [6]:
df_min_order_date = \
    (df_min_order_date
     .assign(date=lambda _df: _df["created_at"] + pd.Timedelta(hours=3, minutes=30))
     .groupby(["user_id"]).agg(
        firstdate=("date", "min")).reset_index()
     .assign(firstdate=lambda _df: _df['firstdate'].dt.normalize())
     .assign(user_id=lambda _df: _df['user_id'].astype('str'))
     )

In [7]:
# add ShamsiDate to dim_date_join
df_dim_date_join = (
    df_dim_date
    .assign(miladi_d=lambda df: pd.to_datetime(df['miladi_d'], errors='coerce'))
    [['miladi_d', 'jalali_1_s']]
    .assign(
        ShamsiDate=lambda df: pd.to_numeric(
            df['jalali_1_s'].astype(str).str.replace('/', ''),
            errors='coerce'
        )
    )
    .rename(columns={'miladi_d': 'date'})
    .drop(columns=['jalali_1_s'])
    [['date', 'ShamsiDate']]
    .assign(date=lambda df: pd.to_datetime(df["date"]).dt.normalize())
)

In [8]:
# add shamsifirstdate to df_min_date
df_min_order_date_clean = \
    (df_min_order_date
     .assign(firstdate=lambda df: pd.to_datetime(df['firstdate']).dt.normalize())
     .merge(df_dim_date_join, left_on="firstdate", right_on="date", how='left')
     .rename(columns={'ShamsiDate': 'shamsifirstdate'})
     .drop(columns=['date'])
     )

In [9]:
# add ShamsiMonth column to df_raw_data
ShamsiMonthRef = [int(f"{year}{month:02}") for year in range(1401, 1410) for month in range(1, 13)]

In [10]:
df_orders = \
    (df_orders
     .assign(date=lambda df: pd.to_datetime(df['date'], errors='coerce'))
     .rename(columns={"_id": "factor_id"})
     .assign(factor_id=lambda df: df['factor_id'].apply(lambda x: str(x) if isinstance(x, ObjectId) else x))
     .drop(columns=["module"])
     .loc[lambda df: df['date'] >= start_train_date_filter]
     .merge(df_dim_date_join, on="date", how="left")
     .assign(ShamsiMonth=lambda df: df['ShamsiDate'].astype(str).str[:6].astype(int))
     )
min_shamsi_month = df_orders['ShamsiMonth'].min()

In [11]:
if int(f"{min_shamsi_month}01") not in df_orders['ShamsiDate'].values:
    idx = ShamsiMonthRef.index(min_shamsi_month)
    min_shamsi_month = ShamsiMonthRef[idx + 1]

max_shamsi_month = df_orders['ShamsiMonth'].max()
idx = ShamsiMonthRef.index(max_shamsi_month)
max_shamsi_month = ShamsiMonthRef[idx - 1]

df_orders = (
    df_orders
    .loc[lambda df: df["ShamsiMonth"] >= min_shamsi_month]
    .loc[lambda df: df["ShamsiMonth"] <= max_shamsi_month]
    .merge(df_min_order_date_clean, on="user_id", how="left")
)

In [12]:
df_orders_with_period = (
    pd.concat(
        [
            df_orders.loc[df_orders["ShamsiMonth"].isin(
                ShamsiMonthRef[idx: idx + window_table_in_months])].assign(
                period=(0 if idx == ShamsiMonthRef.index(min_shamsi_month) else idx - ShamsiMonthRef.index(
                    min_shamsi_month))
            )
            for idx in range(ShamsiMonthRef.index(min_shamsi_month),
                             len(ShamsiMonthRef) - window_table_in_months + 1)
            if (
            (lambda wd: len(wd) > 0 and wd["ShamsiMonth"].nunique() == window_table_in_months)(
                df_orders.loc[
                    df_orders["ShamsiMonth"].isin(ShamsiMonthRef[idx: idx + window_table_in_months])]
            )
        )
        ],
        ignore_index=True
    )
)

In [13]:
# calculate average days between factors index
df_orders_with_period = (
    df_orders_with_period
    .sort_values(by=["user_id", "period", "date"])
    .assign(
        days_diff=lambda df: df.groupby(["user_id", "period"])["date"].diff().dt.days
    )
    .assign(
        avg_days_between_factors=lambda df: df.groupby(["user_id", "period"])["days_diff"].transform("mean")
    )
)

In [14]:
# add NewUserLabelAcc to df_raw_data_with_period for new users but new users labeled
period_ranges = (
    df_orders_with_period
    .groupby("period")["ShamsiDate"]
    .agg(["min", "max"])
    .reset_index()
    .rename({"min": "min_date", "max": "max_date"})
)

In [15]:
df_orders_with_period = (
    df_orders_with_period
    .merge(
        df_orders_with_period
        .groupby("period")["ShamsiDate"]
        .agg(["min", "max"])
        .reset_index()
        .rename(columns={"min": "min_date", "max": "max_date"}),
        on="period", how="left"
    )
    .assign(
        NewUserLabelAcc=lambda df: (
                (df["shamsifirstdate"] >= df["min_date"]) &
                (df["shamsifirstdate"] <= df["max_date"])
        ).astype(int)
    )
)


# RFM Calculation

In [16]:
# Function to calculate RFM for each monthly period
def calc_rfm_monthly(period, df_orders):
    data_window = df_orders[df_orders["period"] == period].copy()

    if data_window.empty:
        return pd.DataFrame()

    grouped = (
        data_window.groupby("user_id", as_index=False)
        .agg(
            max_date=('date', 'max'),
            F=('factor_id', pd.Series.nunique),
            M=('total_payment_price', 'sum')
        )
    )

    end_date = data_window["date"].max()
    grouped["R"] = (end_date - grouped["max_date"]).dt.days + 1

    # grouped["R_Score_raw"] = pd.qcut(grouped["R"], q=5, labels=False, duplicates='drop') + 1

    grouped['R_Score_raw'] = pd.cut(
    grouped['R'],
    bins=[0, 15, 30, 45, 60, float('inf')],
    labels=[1, 2, 3, 4, 5],
    right=True
    ).astype(int)

    grouped["R_Score"] = 6 - grouped["R_Score_raw"]
    grouped["F_Score"] = pd.qcut(grouped["F"], q=20, labels=False, duplicates='drop') + 1
    grouped["M_Score"] = pd.qcut(grouped["M"], q=5, labels=False, duplicates='drop') + 1

    grouped["period"] = period
    grouped["start_month"] = data_window["ShamsiMonth"].min()
    grouped["end_month"] = data_window["ShamsiMonth"].max()

    grouped = grouped.drop(columns=["R_Score_raw"])

    return grouped

In [17]:
# calc df_raw_data_with_period_NULAcc column
df_orders_with_period_NULAcc = (
    df_orders_with_period
    [['user_id', 'period', 'shamsifirstdate', 'NewUserLabelAcc']]
    .drop_duplicates()
)

In [18]:
# Calculate init score_table dataframe
df_score_table = (
    pd.concat(
        [calc_rfm_monthly(p, df_orders_with_period)
         for p in sorted(df_orders_with_period["period"].unique())],
        ignore_index=True
    )
    .assign(
        RFMSeries=lambda df: (
                df["R_Score"].astype(str) +
                df["F_Score"].astype(str) +
                df["M_Score"].astype(str)
        ).astype(int)
    )
    .pipe(lambda df: df.merge(rfm_labels, on="RFMSeries", how="left"))
    .merge(df_orders_with_period_NULAcc, on=["user_id", "period"], how="left")
    .assign(
        clusters_labelFa=lambda df: np.where(
            (df["clusters_labelFa"].isin(["مشتریان جدید", "مشتریان جدید با پتانسیل رشد", "مشتریان جدید در معرض خطر", "مشتریان جدید و ارزشمند در آستانه ریزش", "مشتریان جدید یا جدید ارزشمند و در معرض خطر"])) &
            (df["NewUserLabelAcc"] == 0),
            "مشتریان بازگشته",
            df["clusters_labelFa"]
        )
    )
)

# Add Previous Period Cluster Labels

In [19]:
# Define rfm_clusters
df_rfm_clusters = df_score_table[["user_id", "period", "RFMSeries", "clusters_labelFa", "clusters_inds"]].copy()

In [20]:
df_rfm_clusters_prev = (
    df_score_table[["user_id", "period", "RFMSeries", "clusters_labelFa", "clusters_inds"]]
    .assign(period=lambda df: df["period"] + 1)
    .rename(columns={"clusters_labelFa": "prev_clusters_labelFa"})
)

In [21]:
# add previous clusters label to current
df_rfm_clusters = (df_rfm_clusters
                .merge(df_rfm_clusters_prev[["user_id", "period", "prev_clusters_labelFa"]],
                       on=["user_id", "period"],
                       how="left")
                )

In [22]:
df_rfm_table = df_score_table[
    ["user_id", "R", "F", "M", "period", "clusters_labelFa", "clusters_inds", "RFMSeries"]].copy()

# ✅ Add prev_clusters_labelFa to rfm_table too
df_rfm_table = df_rfm_table.merge(
    df_rfm_clusters[["user_id", "period", "prev_clusters_labelFa"]],
    on=["user_id", "period"],
    how="left"
)

# calc dist, speed , acceleration based on R, F, M scores
df_cluster_means = (
    df_score_table
    .query('clusters_labelFa != "مشتریان بازگشته"')
    .groupby(["period", "clusters_labelFa"]) # to avoid distortion of means by new customers
    .agg(R_mean=("R_Score", "mean"),
         F_mean=("F_Score", "mean"),
         M_mean=("M_Score", "mean"))
    .reset_index()
)

# Add dist, speed, acceleration and their ranks

In [23]:
# 1) expand to long form (each user vs each possible target cluster in same period)
df_score_with_targets = (
    df_score_table
    .merge(
        df_cluster_means.rename(columns={"clusters_labelFa": "target_clusters_labelFa"}),
        on="period",
        how="left"
    )
)

df_score_with_targets = df_score_with_targets.assign(
    dist=lambda df: (
                            (df["R_Score"] - df["R_mean"]) ** 2
                            + (df["F_Score"] - df["F_mean"]) ** 2
                            + (df["M_Score"] - df["M_mean"]) ** 2
                    ) ** 0.5
)

df_score_with_targets = df_score_with_targets.sort_values(
    ["user_id", "target_clusters_labelFa", "period"]
)

df_score_with_targets = df_score_with_targets.assign(
    speed=lambda df: df.groupby(
        ["user_id", "target_clusters_labelFa"]
    )["dist"].diff(),
    acc=lambda df: df.groupby(
        ["user_id", "target_clusters_labelFa"]
    )["speed"].diff()
)

In [24]:
df_score_with_targets_temp = df_score_with_targets.copy()

# ---------------- STEP 1: Unique Table For (user_id, period) ----------------
df_user_period_labels = (
    df_score_with_targets_temp.sort_values(["user_id", "period"])
    .groupby(["user_id", "period"], as_index=False)
    .first()[["user_id", "period", "clusters_labelFa"]]
)

In [25]:
df_user_period_labels = df_user_period_labels.sort_values(["user_id", "period"])

df_user_period_labels["prev_label_1"] = (
    df_user_period_labels.groupby("user_id")["clusters_labelFa"].shift(1)
)
df_user_period_labels["prev_label_2"] = (
    df_user_period_labels.groupby("user_id")["clusters_labelFa"].shift(2)
)


In [26]:
# ---------------- STEP 2: merge prev_label long ----------------
df_score_with_targets_temp = df_score_with_targets_temp.merge(
    df_user_period_labels[["user_id", "period", "prev_label_1", "prev_label_2"]],
    on=["user_id", "period"],
    how="left"
)

# ---------------- DISTANCE RANK ----------------
df_score_with_targets_temp["rank_dist"] = (
    df_score_with_targets_temp.groupby(["period", "clusters_labelFa", "target_clusters_labelFa"])["dist"]
    .rank(method="dense", ascending=True, na_option="bottom")
)

# ---------------- SPEED RANK ----------------
mask_speed = (df_score_with_targets_temp["prev_label_1"] == df_score_with_targets_temp["clusters_labelFa"]) & df_score_with_targets_temp["speed"].notna()
df_score_with_targets_temp["rank_speed"] = pd.NA
df_score_with_targets_temp.loc[mask_speed, "rank_speed"] = (
    df_score_with_targets_temp.loc[mask_speed]
    .groupby(["period", "clusters_labelFa", "target_clusters_labelFa"])["speed"]
    .rank(method="dense", ascending=True, na_option="bottom")
)

# ---------------- ACCELERATION RANK ----------------
mask_acc = (
        (df_score_with_targets_temp["prev_label_1"] == df_score_with_targets_temp["clusters_labelFa"]) &
        (df_score_with_targets_temp["prev_label_2"] == df_score_with_targets_temp["clusters_labelFa"]) &
        df_score_with_targets_temp["acc"].notna()
)
df_score_with_targets_temp["rank_acc"] = pd.NA
df_score_with_targets_temp.loc[mask_acc, "rank_acc"] = (
    df_score_with_targets_temp.loc[mask_acc]
    .groupby(["period", "clusters_labelFa", "target_clusters_labelFa"])["acc"]
    .rank(method="dense", ascending=True, na_option="bottom")
)

# ---------------- TYPE CAST ----------------
for col in ["rank_dist", "rank_speed", "rank_acc"]:
    df_score_with_targets_temp[col] = df_score_with_targets_temp[col].astype("Int64")

df_score_with_targets = df_score_with_targets.drop_duplicates()

df_score_with_targets = df_score_with_targets_temp


In [27]:
df_orders_with_period_merge = (
    df_orders_with_period[["period", "ShamsiMonth"]]
    .drop_duplicates()
    .groupby("period")
    .agg(max_shamsi_month=("ShamsiMonth", "max"))
    .reset_index()
)

df_score_with_targets = (
    df_score_with_targets
    .merge(
        df_orders_with_period_merge[["period", "max_shamsi_month"]],
        on="period",
        how="left"
    )
    .rename(columns={"max_shamsi_month": "ShamsiMonth"})
)

# Add 4 New Clusters and Fill NULL values and returned customers

In [ ]:
max_rfm_lables_index = int(rfm_labels.clusters_inds.max())
cluster_lables_additional_indices = {
'مشتریان بازگشته ارزشمند': max_rfm_lables_index + 1,
'مشتریان بازگشته گذری': max_rfm_lables_index + 2,
'مشتریان از دست رفته ارزشمند': max_rfm_lables_index + 3,
'مشتریان از دست رفته گذری': max_rfm_lables_index + 4,
'موقت گذری' : max_rfm_lables_index + 5,
'موفت ارزشمند' : max_rfm_lables_index + 6
}

In [ ]:
unique_target_clusters = rfm_labels['clusters_labelFa'].unique()
all_periods = sorted(df_score_with_targets['period'].unique())

In [ ]:
# Find First Period with all clusters
user_first_period = \
    (
        df_score_with_targets
        .groupby('user_id')[['period']]
        .min()
        .reset_index()
        .rename(columns={'period': 'first_period'})
    )

In [ ]:
# Create complete skeleton: every user × every period (after their first) × every target cluster
df_complete_skeleton = []

for _, row in user_first_period.iterrows():
    user_id = row['user_id']
    first_period = row['first_period']

    # Get all periods from first_period onwards
    relevant_periods = [p for p in all_periods if p >= first_period]

    # Create records for this user
    for period in relevant_periods:
        for target_cluster in unique_target_clusters:
            df_complete_skeleton.append({
                'user_id': user_id,
                'period': period,
                'target_clusters_labelFa': target_cluster,
            })

df_complete_skeleton = pd.DataFrame(df_complete_skeleton)

In [ ]:
df_complete_skeleton_first_phase_assignment = \
(
    df_complete_skeleton
    .merge(
        df_score_with_targets,
        on=['user_id', 'period', 'target_clusters_labelFa'],
        how='left'
    )
)

In [ ]:
# First, let's create a helper dataframe for period-level values
df_period_values = (
    df_complete_skeleton_first_phase_assignment
    .groupby('period')
    .agg(
        start_month=('start_month', 'first'),
        end_month=('end_month', 'first'),
        ShamsiMonth=('ShamsiMonth', 'first')
    )
    .reset_index()
)

In [ ]:
# Sort by user_id and period for proper shifting
df_complete_skeleton_filled = (
    df_complete_skeleton_first_phase_assignment
    .sort_values(['user_id', 'period'])
    .copy()
)

# Fill period-level columns first
df_complete_skeleton_filled = df_complete_skeleton_filled.merge(
    df_period_values,
    on='period',
    how='left',
    suffixes=('', '_fill')
)